In [1]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from analysis import read_behaviors, read_moea_fitness, unpickle_moea
from utility.util_functions import calc_str_disparity, euclidean_distance
from utility.visualize import plot_path, plot_moea_fitness, plot_surrogate_model, plot_str_disparity, plot_moea_fitness_2d
import pickle
import numpy as np
from deap import base, creator
from evolution.simulation import Disparity
import scipy as sp
from scipy import stats
import math
init_notebook_mode(connected=True)


ModuleNotFoundError: DBus works only on linux!


Using TensorFlow backend.


ModuleNotFoundError: DBus works only on linux!


In [2]:
# get the path for all behavioral features
files = sorted(Path('data/results/').glob('transferability_*/behavioral_features.dat'))
fitness_f = sorted(Path('data/results/').glob('transferability_*/fitness.dat'))

In [3]:
columns = [
        'avg_left', 'avg_right',
        's1', 's2', 's3', 's4', 's5', 's6', 's7',
        'area0_percentage',
        'area1_percentage',
        'area2_percentage',
    ]

In [4]:
fitness_f[9]

PosixPath('data/results/transferability_simulation_18/fitness.dat')

In [5]:
behaviors = read_behaviors([files[9]])[0]

In [6]:
behaviors.loc[behaviors['genome_id'] == 494].head()

,gen,genome_id,simulation,avg_left,avg_right,s1,s2,s3,s4,s5,s6,s7,area0_percentage,area1_percentage,area2_percentage
671,24,494,VREP,0.634055,0.633922,0.806828,0.776853,0.000000,0.000000,0.000000,0.004996,0.004996,1.000000,0.0,0.000000
683,24,494,THYMIO,0.624684,0.915316,0.000000,0.012717,0.011561,0.745665,0.811561,0.000000,0.000000,0.567698,0.0,0.432302


In [7]:
behaviors.head()

,gen,genome_id,simulation,avg_left,avg_right,s1,s2,s3,s4,s5,s6,s7,area0_percentage,area1_percentage,area2_percentage
0,0,1,VREP,0.220288,0.040265,0.0,0.0,0.0,0.0,0.0,0.987179,0.987179,1.0,0.0,0.0
1,0,1,THYMIO,0.261485,0.047720,0.0,0.0,0.0,0.0,0.0,0.157895,0.289474,1.0,0.0,0.0
2,0,1,VREP,0.220288,0.040265,0.0,0.0,0.0,0.0,0.0,0.987179,0.987179,1.0,0.0,0.0
3,0,2,VREP,0.676044,0.652744,0.0,0.0,0.0,0.0,0.0,0.150000,0.140000,1.0,0.0,0.0
4,0,3,VREP,0.066874,0.514206,0.0,0.0,0.0,0.0,0.0,0.630000,0.310000,1.0,0.0,0.0


In [ ]:
b_grouped = behaviors.groupby(['genome_id']).filter(lambda x: len(x) > 1).groupby(['genome_id', 'simulation']).first()

In [ ]:
b_grouped.head()

In [ ]:
euclidean_distance(b_grouped[columns].iloc[1], b_grouped[columns].iloc[0])

In [ ]:
for i in range(1, len(b_grouped), 2):
    print('genome_id:', 
          b_grouped[columns].iloc[i].name[0],
          'actual diversity',
          euclidean_distance(
              b_grouped[columns].iloc[i],
              b_grouped[columns].iloc[i-1]
          )
    )

In [8]:
fitness_data = read_moea_fitness([fitness_f[9]])[0]

In [ ]:
fitness_data.sort_values(by=['fitness'])

In [ ]:
fitness_data.head()

In [ ]:
hof = unpickle_moea('data/results/transferability_simulation_18/hof')

In [ ]:
len(hof)

In [ ]:
plot_moea_fitness_2d(fitness_data, hof)

In [ ]:
hof_ids = []
for ind in hof:
    hof_ids.append(ind.key)
    print(ind.key, ind.fitness.values)

In [9]:
genomes_file = sorted(Path('data/results/transferability_simulation_18/deap_inds').glob('*_genome.pkl'))
transfered_file = sorted(Path('data/results/transferability_simulation_18/deap_inds').glob('*_transformed_genome_.pkl'))
genomes = [unpickle_moea(genome) for genome in genomes_file]
transferred = [unpickle_moea(genome) for genome in transfered_file]

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [ ]:
transfered_file

In [10]:
len(transfered_file)

224

In [11]:
g = sorted(genomes, key=lambda x: x.key)

In [12]:
t = sorted(transferred, key=lambda x: x.key)

In [13]:
sim_genome = list(filter(lambda x: x.key == 494, g))[0]

In [14]:
print(sim_genome.key, sim_genome.fitness.values, sim_genome.sim_time)

494 (3.3845905856901366, 2.176647353664289, 1.0) 60.049999999998725


In [15]:
trans_genome = list(filter(lambda x: x.key == 494, t))[0]

In [16]:
print(trans_genome.key, trans_genome.fitness.values)

494 (3.3845905856901366, 2.176647353664289, 1.0)


In [17]:
sim_genome.sim_time

60.049999999998725

In [ ]:
sim_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10], g)), key=lambda x: x.fitness.values[0])
trans_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10] t)), key=lambda x: x.fitness.values[0])

In [18]:
plot_path([sim_genome, trans_genome], 'The discrepancy of traveled trajectories in simulation & reality.')
# plot_path(sim_genomes)

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [ ]:
sim_genome.position

In [ ]:
trans_genome.position

In [ ]:
plot_path(trans_genomes)

In [19]:
calc_str_disparity(trans_genome.position, sim_genome.position)

26.59650042033296

In [20]:
euclidean_distance(trans_genome.position, sim_genome.position)

1.9587116144317558

In [ ]:
plot_surrogate_model(fitness_data)

In [21]:
# merge  
union_t = [(x, y) for x in g for y in t if x.key == y.key]
for s, t in union_t:
    np.array_equal(s, t)
    assert s.key == t.key

In [ ]:
str_disparity = Disparity()

In [ ]:
for s, t in union_t:
    print(s.gen, end=" ")

In [24]:
str_disparities = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_t
                  ]    

str_euclidean = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_t
                ]

In [ ]:
sorted(str_disparities, key=lambda x: x[0])

In [ ]:
np.array([str_disparity[1] for str_disparity in str_disparities])
np.array([real_disparity[2] for real_disparity in str_disparities])

filter_disparities = list(filter(lambda x: x[2] < 5, str_disparities))
filter_disparities[:4]

In [ ]:
plot_str_disparity(str_disparities)

In [ ]:
actual = []
approximated = []
for s, t in union_t:
    approximated.append(s.fitness.values[1])
    actual.append(calc_str_disparity(t.position, s.position))

In [ ]:
stats.pearsonr(actual, approximated)

In [25]:
str_euclidean

[(1, 27, 4.511122271805817, 0.22083602115838719, 2.1618874243556885),
 (1, 31, 5.20168203033889, 0.22083602115838719, 0.8704884585817774),
 (2, 47, 3.7269785681007557, 0.9817640118427502, 1.5069315329509638),
 (2, 54, 5.828766334129683, 0.9817640118427502, 9.18696947953385),
 (3, 62, 4.6134875990238475, 2.8605492079707773, 4.704113996889614),
 (3, 68, 3.139842704784066, 2.8605492079707773, 1.8261312366998856),
 (4, 81, 5.56052992240503, 2.902637143456986, 1.18386775617735),
 (4, 91, 5.787573357303519, 2.902637143456986, 0.6085015628976186),
 (4, 95, 5.640007221236081, 2.902637143456986, 0.680036914224531),
 (4, 99, 5.595102751384721, 2.902637143456986, 0.8144829665065617),
 (5, 103, 4.228242667407882, 1.6954398914786477, 4.234229194937672),
 (5, 111, 7.451002883058336, 1.6954398914786477, 2.0757833340132246),
 (5, 112, 5.378864849673304, 1.6954398914786477, 3.5113258114605532),
 (5, 114, 5.593293545309893, 1.6954398914786477, 0.7465298672042195),
 (5, 117, 7.466823614311536, 1.69543989

In [ ]:
distance = 0
for i in range(0, len(trans_genome.position)-1):
    distance += math.sqrt(((sim_genome.position[i][0] - 
                            sim_genome.position[i][1])**2) + 
                          ((sim_genome.position[i+1][1] - 
                            sim_genome.position[i+1][1])**2))

In [ ]:
def traveled_distance(genome):
    distance = 0
    for i in range(0, len(genome.position)-1):
        distance += math.sqrt(((genome.position[i][0] - 
                                genome.position[i][1])**2) + 
                              ((genome.position[i+1][1] - 
                                genome.position[i+1][1])**2))
    return distance

In [ ]:
traveled_distance(trans_genome)

In [ ]:
traveled_distance(sim_genome)